In [250]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import xlsxwriter

In [251]:
df = pd.read_excel("../data/zadatak 2.xlsx", decimal = ",")
df.loc[:, "Datum"] = pd.to_datetime(df.loc[:, "Datum"])

In [252]:
df.head()

,POSTAJA,šifra,Datum,Temperatura vode (°C),pH vrijednost,Električna vodljivost (μS/cm),Ukupne suspendirane tvari (mg/l),Alkalitet m-vrijednost (mgCaCO₃/l),Tvrdoća ukupna (mgCaCO₃/l),Otopljeni kisik (mgO₂/l),...,TOC (mg/l),DOC (mg/l),Kalcij (mg/l),Magnezij (mg/l),Natrij (mg/l),Kalij (mg/l),Kloridi (mg/l),Fluoridi (mg/l),Sulfati (mg/l),"Silikati, otopljeni (mgSiO₂/l)"
NaN,"Sava, Drenje-Jesenice",SA1,2015-01-19,8.3,8.1,436.0,12.5,202.0,222.4,10.7,...,1.07,1.04,65.9,14.0,6.39,1.13,9.04,0.0360,13.0,3.76
NaN,"Sava, Drenje-Jesenice",SA1,2015-02-18,7.6,8.2,466.0,4.0,213.0,235.3,12.0,...,1.20,1.09,69.7,14.8,8.01,1.24,12.73,0.0300,13.2,4.10
NaN,"Sava, Drenje-Jesenice",SA1,2015-10-03,7.7,8.1,436.0,3.6,207.0,229.0,11.6,...,1.04,0.97,67.3,14.7,5.37,0.97,8.05,0.0260,11.3,3.53
NaN,"Sava, Drenje-Jesenice",SA1,2015-04-15,12.7,8.2,423.0,4.0,195.0,213.8,10.6,...,1.26,1.17,62.3,14.1,5.50,1.13,7.66,0.0210,12.9,2.64
NaN,"Sava, Drenje-Jesenice",SA1,2015-12-05,16.8,8.4,378.0,11.6,172.0,197.0,11.5,...,1.32,1.13,53.5,15.3,6.25,1.11,8.42,0.0125,13.8,0.53


In [253]:
fizKemPok = list(df.columns)[3:9]
rezKisika = list(df.columns)[9:13]
hranjTvar = list(df.columns)[13:22]
metali = list(df.columns)[22:24]
orgSpoj = list(df.columns)[24:26]
ioni = list(df.columns)[26:]

noviIndex = (["Pokazatelji"]
             + ["Fizikalno kemijski pokazatelji"]
             + fizKemPok
             + [""]
             + ["Režim kisika"]
             + rezKisika
             + [""]
             + ["Hranjive tvari"]
             + hranjTvar
             + [""]
             + ["Metali"]
             + metali
             + [""]
             + ["Organski spojevi"]
             + orgSpoj
             + [""]
             + ["Ioni"]
             + ioni)

In [254]:
noviIndex

['Pokazatelji',
 'Fizikalno kemijski pokazatelji',
 'Temperatura vode (°C)',
 'pH vrijednost',
 'Električna vodljivost (μS/cm)',
 'Ukupne suspendirane tvari (mg/l)',
 'Alkalitet m-vrijednost (mgCaCO₃/l)',
 'Tvrdoća ukupna (mgCaCO₃/l)',
 '',
 'Režim kisika',
 'Otopljeni kisik (mgO₂/l)',
 'Zasićenje kisikom (%)',
 'BPK₅ (mgO₂/l)',
 'KPK-Mn (mgO₂/l)',
 '',
 'Hranjive tvari',
 'Amonij (mgN/l)',
 'Nitriti (mgN/l)',
 'Nitrati (mgN/l)',
 'Neionizirani amonijak (mgNH₃/l)',
 'Ukupni dušik (mgN/l)',
 'Anorganski dušik (mgN/l)',
 'Organski dušik (mgN/l)',
 'Ortofosfati (mgP/l)',
 'Ukupni fosfor (mgP/l)',
 '',
 'Metali',
 'Bakar, otopljeni (μgCu/l)',
 'Cink, otopljeni (μgZn/l)',
 '',
 'Organski spojevi',
 'TOC (mg/l)',
 'DOC (mg/l)',
 '',
 'Ioni',
 'Kalcij (mg/l)',
 'Magnezij (mg/l)',
 'Natrij (mg/l)',
 'Kalij (mg/l)',
 'Kloridi (mg/l)',
 'Fluoridi (mg/l)',
 'Sulfati (mg/l)',
 'Silikati, otopljeni (mgSiO₂/l)']

In [255]:
godine = list(df.loc[:, "Datum"].apply(lambda x: x.year).unique())

In [256]:
for i in df.loc[:, "šifra"].unique():
    assert(len(df.loc[df.loc[:, "šifra"] == i, "POSTAJA"].unique()) == 1)

In [257]:
postaje = {i: df.loc[df.loc[:, "šifra"] == i, "POSTAJA"].unique(
    )[0] for i in df.loc[:, "šifra"].unique()}

In [258]:
testGodina = "2016"
testPostaja = "DA2"
testFrame = df.loc[(df.loc[:, "šifra"] == testPostaja)
                   & (df.loc[:, "Datum"].apply(lambda x: x.year).astype("str") == testGodina)]

In [259]:
def crunch(frame):
    outFrame = frame.count().to_frame()
    outFrame.columns = ["br.an."]
    outFrame.loc[:, "MAX"] = frame.max()
    outFrame.loc[:, "MIN"] = frame.min()
    outFrame.loc[:, "SR.VR."] = frame.mean()
    outFrame.loc[:, "ST.DEV."] = frame.std()
    outFrame.loc[:, "10%"] = frame.quantile(.1)
    outFrame.loc[:, "50%"] = frame.quantile(.5)
    outFrame.loc[:, "90%"] = frame.quantile(.9)
    
    return outFrame

def sliceAndDice(frame, noviIndex = noviIndex):
    outFrame = pd.DataFrame(index = pd.Index(noviIndex),
                             columns = frame.columns)
    outFrame.loc["Pokazatelji"] = frame.columns
    for i in frame.index:
        outFrame.loc[i] = frame.loc[i]
    
    return outFrame.reset_index()

In [260]:
crunchFrame = crunch(testFrame.drop(["POSTAJA", "Datum", "šifra"], axis = 1))
sliceFrame = sliceAndDice(crunchFrame)

In [280]:
writer = pd.ExcelWriter("../data/rezultat2.xlsx",
                        engine = "xlsxwriter")

counter = 0

for s, p in postaje.items():
    for g in ["2015", "2016"]:
        
        if (s == "SA1"):
            continue
        
        shift = counter * 53
        
        #inFrame = df.loc[(df.loc[:, "šifra"] == s)
        #           & (df.loc[:, "Datum"].apply(lambda x: x.year).astype("str") == str(g))]
        inFrame = testFrame
        
        crunchFrame = crunch(inFrame.drop(["POSTAJA", "Datum", "šifra"], axis = 1))
        sliceFrame = sliceAndDice(crunchFrame)
        
        sliceFrame.to_excel(writer, sheet_name="rezultati", index=False,
                           header = False, startrow = 7 + shift)
        
        if counter == 0:
            workbook = writer.book
            workbook.strings_to_numbers = True
        
            normalFormat = workbook.add_format({"font_name":"Ariel", "font_size":10})
            subTitleFormat = workbook.add_format({"font_name":"Ariel", "font_size":10, "bold": True})
            headerFormat = workbook.add_format({"font_name":"Ariel", "font_size":14, "bold": True})

            worksheet = writer.sheets["rezultati"]
            
            worksheet.set_column(1, 8, 10, normalFormat)
            worksheet.set_column(0, 0, 40, normalFormat)

        worksheet.write(0 + shift, 0, "STATISTIKA")
        worksheet.write(1 + shift, 5, "Naziv postaje: {}".format(p))
        worksheet.write(2 + shift, 5, "Šifra postaje: {}".format(p) )
        worksheet.write(3 + shift, 5, "Razdoblje: 01.01.{}-30.12.{}".format(g, g))
        worksheet.write(4 + shift, 5, "Medij: Površinska voda")
        worksheet.write(5 + shift, 5, "Mikrolokacija:")
        worksheet.write(6 + shift, 0, "Datum ispisa: 18.10.2018")


        worksheet.set_row(0 + shift, None, headerFormat)
        worksheet.set_row(7 + shift, None, subTitleFormat)
        worksheet.set_row(8 + shift, None, subTitleFormat)
        worksheet.set_row(16 + shift, None, subTitleFormat)
        worksheet.set_row(22 + shift, None, subTitleFormat)
        worksheet.set_row(33 + shift, None, subTitleFormat)
        worksheet.set_row(37 + shift, None, subTitleFormat)
        worksheet.set_row(41 + shift, None, subTitleFormat)
        
        counter += 1
writer.save()
print("done, counter: {}".format(counter))

done, counter: 90
